In [1]:
import sys

In [2]:
!conda install --yes --prefix {sys.prefix} -c anaconda nltk
!conda install --yes --prefix {sys.prefix} -c anaconda gensim

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: ...working... done

## Package Plan ##

  environment location: D:\Anaconda3

  added / updated specs:
    - gensim


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    boto3-1.9.234              |             py_0          91 KB  anaconda
    botocore-1.12.234          |             py_0         3.4 MB  anaconda
    bz2file-0.98               |           py37_1          13 KB  anaconda
    gensim-3.4.0               |   py37hfa6e2cd_0        21.4 MB  anaconda
    jmespath-0.9.4             |             py_0          22 KB  anaconda
    s3transfer-0.2.1           |   

In [8]:
!conda install --yes --prefix {sys.prefix} -c conda-forge pyldavis

Solving environment: ...working... done

## Package Plan ##

  environment location: D:\Anaconda3

  added / updated specs:
    - pyldavis


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.7.12               |           py37_0         3.0 MB  conda-forge
    funcy-1.13                 |             py_0          28 KB  conda-forge
    pyldavis-2.1.2             |             py_0          81 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.1 MB

The following NEW packages will be INSTALLED:

  funcy              conda-forge/noarch::funcy-1.13-py_0
  pyldavis           conda-forge/noarch::pyldavis-2.1.2-py_0

The following packages will be SUPERSEDED by a higher-priority channel:

  conda                                            anaconda --> conda-forge




pyldavis-2.1.2       | 81 KB   

In [1]:
from pathlib import Path

keyFolder = Path("C://develop/keys/social_media_analytics/")

redditKeyFile = keyFolder / "reddit.txt"

with open(redditKeyFile) as file:
    content = file.readlines()

clientId = content[0].split(':')[1].strip()
clientSecret = content[1].split(':')[1].strip()
password = content[2].split(':')[1].strip()
username = content[3].split(':')[1].strip()

In [2]:
# get nltk

import os
# uncomment to download nltk library
#nltk.download()
os.environ["NLTK_DATA"] = "D:\nltk_data"

In [3]:
# imports

import praw
import json
import gensim

from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
from gensim import corpora, models, similarities

D:\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [23]:
# get intial post information and do parsing

stop_words = set(stopwords.words('english'))
stop_words.update(['.',  ',', '"', "'", "’", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}']) # remove it if you need punctuation

file = open("redditprocess.txt", "w")
r = praw.Reddit(client_id=clientId,
                client_secret=clientSecret,
                password=password,
                user_agent='testScript by /u/rsussan',
                username=username)

for post in r.subreddit('aquariums') \
             .comments(limit=1000):
    try:
        file.write(str(post.body) + '\n')
    except:
        # swallow the exception, there must be some character in this post that breaks the write
        print(sys.exc_info()[0])
    words = [i.lower() for i in wordpunct_tokenize(post.body) if i.lower() not in stop_words]
    all_tokens = ' '.join(words)

file.close()

# Remove words that appear only once

texts = words
# remove words that appear only once
#all_tokens = sum(texts, [])
tokens_once = set(words for words in set(all_tokens) if all_tokens.count(words) == 1)
texts = [[words for words in texts if words not in tokens_once]
         for words in all_tokens]


<class 'UnicodeEncodeError'>
<class 'UnicodeEncodeError'>
<class 'UnicodeEncodeError'>
<class 'UnicodeEncodeError'>
<class 'UnicodeEncodeError'>
<class 'UnicodeEncodeError'>
<class 'UnicodeEncodeError'>
<class 'UnicodeEncodeError'>
<class 'UnicodeEncodeError'>
<class 'UnicodeEncodeError'>
<class 'UnicodeEncodeError'>
<class 'UnicodeEncodeError'>
<class 'UnicodeEncodeError'>
<class 'UnicodeEncodeError'>
<class 'UnicodeEncodeError'>
<class 'UnicodeEncodeError'>
<class 'UnicodeEncodeError'>
<class 'UnicodeEncodeError'>
<class 'UnicodeEncodeError'>
<class 'UnicodeEncodeError'>
<class 'UnicodeEncodeError'>
<class 'UnicodeEncodeError'>
<class 'UnicodeEncodeError'>
<class 'UnicodeEncodeError'>
<class 'UnicodeEncodeError'>
<class 'UnicodeEncodeError'>


In [24]:
#Setup gensim dictionary

dictionary = corpora.Dictionary(texts)

corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize('reddit.mm', corpus)

lda = gensim.models.LdaModel(corpus, id2word=dictionary, alpha='auto', num_topics=10)
for i in lda.show_topics():
    print (i)

D:\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
D:\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
D:\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in

(0, '0.113*"fine" + 0.076*"guppies" + 0.071*"time" + 0.067*"sounds" + 0.063*"shadowy" + 0.062*"sound" + 0.061*"chunk" + 0.055*"along" + 0.055*"spend" + 0.051*"places"')
(1, '0.107*"fine" + 0.065*"guppies" + 0.061*"mine" + 0.060*"sound" + 0.059*"get" + 0.058*"order" + 0.058*"along" + 0.057*"chunk" + 0.057*"bristlenose" + 0.056*"spend"')
(2, '0.059*"fine" + 0.059*"spend" + 0.059*"sounds" + 0.059*"chunk" + 0.059*"shadowy" + 0.059*"fair" + 0.059*"order" + 0.059*"get" + 0.059*"bristlenose" + 0.059*"mine"')
(3, '0.100*"fine" + 0.069*"order" + 0.063*"along" + 0.061*"fair" + 0.060*"mine" + 0.059*"spend" + 0.058*"shadowy" + 0.056*"get" + 0.056*"behaviour" + 0.056*"chunk"')
(4, '0.115*"fine" + 0.061*"sounds" + 0.061*"time" + 0.061*"fair" + 0.059*"behaviour" + 0.058*"sound" + 0.057*"places" + 0.057*"shadowy" + 0.056*"get" + 0.056*"spend"')
(5, '0.105*"fine" + 0.060*"along" + 0.059*"order" + 0.058*"spend" + 0.058*"shadowy" + 0.058*"mine" + 0.057*"number" + 0.057*"fair" + 0.057*"bristlenose" + 0.05

In [25]:
import pyLDAvis.gensim

# something right here is not working, figure out why
topic_vis = pyLDAvis.gensim.prepare(lda, corpus, dictionary)

D:\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [26]:
# display the charts

pyLDAvis.display(topic_vis)